# Predicting winner with Deep Learning (Keras) 
(Uses Elo rating as features)
In this simple example, I will use Elo rating's score as one input feature. 

In [27]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
import pickle
import numpy as np

## Step 1: Load dataset
I imported pickle dataframe which I saved on my previous exercise using elo rating to predict the winner. 

In [4]:
with open("elo_df.pkl", 'rb') as fp:
    elo_df = pickle.load(fp)

In [7]:
elo_df[:3]

,win,lose,date,w_elo_before_game,w_elo_after_game,l_elo_before_game,l_elo_after_game,year,month,day,total_days
0,4,12,1/19/2015,1500,1532,1500,1468,2015,1,19,19.0
1,8,1,1/19/2015,1500,1532,1500,1468,2015,1,19,19.0
2,3,0,1/26/2015,1500,1532,1500,1468,2015,1,26,26.0


## Step 2: Prepare x(input) and y(output)
The start of most of Machine Learning problems (except supervised learning) starts with defining x's and y's. For the purpose of this simple example, I have defined: 
#### x
Probabiliy for winning (for those who had higher Elo score pre-match) based on Elo rating & prediction function
#### y
Win/lose. I will treat this as categorical value (1 for win, and 0 for lose)

### Prepare X values
Given a match, I calcualted expected win probability based on elo rating before the match. 

In [10]:
# The following function is from Elo rating notebook. 
# It calculates winning probability of elo_a against elo_b 
def expected_result(elo_a, elo_b):
    expect_a = 1.0/(1+10**((elo_b - elo_a)/elo_width))
    return expect_a

mean_elo = 1500
elo_width = 400
k_factor = 64

In [11]:
elo_a = elo_df["w_elo_before_game"].tolist()
elo_b = elo_df["l_elo_before_game"].tolist()

expected_win = []
for idx, a in enumerate(elo_a):
    expected_a = expected_result(a, elo_b[idx])
    expected_win.append(expected_a)

#### What does it mean?
In match 1~4, before-Elo-score is the same for each contestant. Therfore expected win is also 0.5, which is random. For the 5th game, winner's before-Elo-score is lower than the loser. Based on Elo ranking, winner's wining probability was 40.89%, but won against the odds. 

In [16]:
elo_df[["win", "lose","date", "w_elo_before_game","l_elo_before_game","expected"]][:5]

,win,lose,date,w_elo_before_game,l_elo_before_game
0,4,12,1/19/2015,1500,1500
1,8,1,1/19/2015,1500,1500
2,3,0,1/26/2015,1500,1500
3,14,7,1/26/2015,1500,1500
4,7,8,2/2/2015,1468,1532


In [12]:
expected_win[:5]

[0.5, 0.5, 0.5, 0.5, 0.4089244036850566]

#### Good enough? 
I transformed the result into n-dimensional array so it will be easier to put into Keras model. Plus, I filtered out the game scores with 0.5 predictions. Out of 313 values we started, now we have 304 values left.

In [34]:
print(len(elo_df))
elo_df = elo_df[["win", "lose","date", "w_elo_before_game","l_elo_before_game"]]
elo_df["random_match"] = elo_df["w_elo_before_game"] - elo_df["l_elo_before_game"] 
elo_df_filtered = elo_df.loc[elo_df["random_match"] != 0].reset_index()
print(len(elo_df_filtered))
elo_df_filtered[:3]

313
304


,index,win,lose,date,w_elo_before_game,l_elo_before_game,random_match
0,4,7,8,2/2/2015,1468,1532,-64
1,5,1,3,2/2/2015,1468,1532,-64
2,8,3,1,2/16/2015,1494,1505,-11


In [39]:
# Calculate again since we have a new data frame (filtered)
elo_a = elo_df_filtered["w_elo_before_game"].tolist()
elo_b = elo_df_filtered["l_elo_before_game"].tolist()

expected_win = []
for idx, a in enumerate(elo_a):
    expected_a = expected_result(a, elo_b[idx])
    expected_win.append(expected_a)

In [60]:
# X values in n-dimensional array
x = np.ones((len(elo_df_filtered), 1))
for row in elo_df_filtered.itertuples():
    idx = row.Index
    if expected_win[idx] > 0.5:
        x[idx][0] = expected_win[idx]
    else: 
        x[idx][0] = 1 - expected_win[idx]

In [61]:
x[:10]

array([[ 0.5910756 ],
       [ 0.5910756 ],
       [ 0.51582499],
       [ 0.50863384],
       [ 0.50719508],
       [ 0.67630167],
       [ 0.66099909],
       [ 0.59801377],
       [ 0.59246623],
       [ 0.65582051]])

Now let's check the x'shapes.

In [62]:
print(x.shape)
print(x[0].shape)

(304, 1)
(1,)


### Prepare Y values
First, we need to capture whether the expected winner (person with higher Elo ranking pregame) lost or won.

In [63]:
y_ = []
for row in elo_df_filtered.itertuples():
    idx = row.Index
    if expected_win[idx] > 0.5: # For the winners with higher pre-game Elo ranking
        y_.append(1)
    else: 
        y_.append(0)

y_[:10]

[0, 0, 0, 1, 1, 1, 1, 0, 1, 1]

#### Convert Y with one-hot encoding
There are two ways. First I will use my favorite numpy.

In [64]:
y_ohe = np.zeros((len(y_), 2),dtype=np.int8)
y_ohe[np.arange(len(y_)), y_] = 1
y = y_ohe
print(y.shape)
print(y[0].shape)
print(y[:3])

(304, 2)
(2,)
[[1 0]
 [1 0]
 [1 0]]


Another way is to use Kera's util function.

In [65]:
y_categorical = np_utils.to_categorical(y_, 2)
print(y_categorical.shape)
print(y_categorical[0].shape)
print(y_categorical[:3])

(304, 2)
(2,)
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


In [66]:
y = y_categorical

#### Divide the dataset into trian & test dataset

In [69]:
ratio = 0.85
x_train = np.array(x[:int((len(x)*ratio))])
x_test = np.array(x[len(x_train):])
y_train = np.array(y[:int((len(y)*ratio))])
y_test = np.array(y[len(y_train):])

In [70]:
print(len(x_train))
print(len(x_test))

258
46


### Yes. This is only a toy example, so the dataset is tiny. 
I don't expect neural network to do much here. Although I want to see if Keras' neural network model can be built upon NBH dataset.

## Step 3: Define model architecture
The simplest model is defined in the Sequential class which is a linear stack of Layers.

In [72]:
# Declare Sequential model
model = Sequential()

In [77]:
# Next, declare input layer. Input shape parameter is the shape of 1 sample.
model.add(Dense(512, input_shape=(1,)))
model.add(Activation('relu')) 
# An "activation" is just a non-linear function applied to the output of the layer above.

# If I print, the current shape of the model output, it will return me this.
print(model.output_shape)

(None, 512)


In [78]:
# Now add more layers.
# We want to add Dropout layer to regularize the model in order to prevent overfitting.
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2)) # Corresponds to the final output size of 2.
model.add(Activation('softmax'))

This special "softmax" activation among other things, ensures the output is a valid probaility distribution, that its values are all non-negative and sum to 1.

## Step 4: Compile model
Cannot believe it is almost done already. The hard part is already over and I will just compile the model.

In [85]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Step 5: Fit model on training data

In [86]:
model.fit(x_train, y_train,
          batch_size=32, epochs=4,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 258 samples, validate on 46 samples
Epoch 1/4
258/258 [==============================] - 1s 2ms/step - loss: 0.6942 - val_loss: 0.6997
Epoch 2/4
258/258 [==============================] - 0s 514us/step - loss: 0.6925 - val_loss: 0.7088
Epoch 3/4
258/258 [==============================] - 0s 498us/step - loss: 0.6991 - val_loss: 0.7147
Epoch 4/4
258/258 [==============================] - 0s 527us/step - loss: 0.6939 - val_loss: 0.6986


Generally speaking, the lower the loss, the better a model. The loss is calculated on training and validation and its interperation is how well the model is doing for these two sets. Unlike accuracy, loss is not a percentage. It is a summation of the errors made for each example in training or validation sets.

### That was easy. huh?

## Step 6: Evaluate model on test data

In [81]:
score = model.evaluate(x_test, y_test, verbose = 1)
score

46/46 [==============================] - 0s 361us/step


0.71154286291288293

Score is the mean of the loss for each test sample. 

### Accuracy
The predict_classes function outputs the highest probability class according to the trained classifier for each input example.

In [109]:
prediction = model.predict(x_test)
predicted_classes = model.predict_classes(x_test)

In [129]:
def categorical_accuracy(y_true, y_pred):
    accu = 0
    for idx, y in enumerate(y_true):
        y_t = 0
        if y[0] == 1:
            pass
        else:
            y_t += 1
            
        if y_t == y_pred[idx]:
            accu += 1
    accuracy = accu / len(y_true)
    print("Accuracy: ", accuracy)
    return accuracy

In [130]:
accuracy = categorical_accuracy(y_test,predicted_classes)

Accuracy:  0.43478260869565216


Okay. I knew it wasn't going to be great, but it looks like a monkey could have done better!
### But it gives a great insight: One chef's winning streak can't last forever.
Elo ranking is based on the winner's past performance only. It assumes that if you have one in the past, then you will win again. Given the fact that our prediciton is below 50%, it means, anyone can cook (and win).